In [3]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
from heapq import nlargest
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist


In [4]:
def getTextWaPo(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text,"lxml")
    text = ' '.join(map(lambda p: p.text, soup.find_all('article')))
    return text.encode('ascii', errors='replace').decode().replace("?"," ")



In [5]:
articleURL = "https://www.washingtonpost.com/news/the-switch/wp/2016/10/18/the-pentagons-massive-new-telescope-is-designed-to-track-space-junk-and-watch-out-for-killer-asteroids/"
text = getTextWaPo(articleURL)



In [6]:
def summarize(text, n): # text and number of sentences to be included in our summary
    sents = sent_tokenize(text)
    
    assert n <= len(sents) #check if the text has required number of sentences
    word_sent = word_tokenize(text.lower())
    _stopwords = set(stopwords.words('english') + list(punctuation))
    
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    
    
    ranking = defaultdict(int)
    
    for i,sent in enumerate(sents):
        for w in word_tokenize(sent.lower()):
            if w in freq:
                ranking[i] += freq[w]
             
        
    sents_idx = nlargest(n, ranking, key=ranking.get)
    return [sents[j] for j in sorted(sents_idx)]


In [8]:
summarize(text,4)

['On Tuesday, the Defense Department took another significant step toward monitoring all of the cosmic junk swirling around in space, by delivering a gigantic new telescope capable of seeing small objects from very far away.',
 'The telescope is  a big improvement over the legacy ground-based optical telescopes that are used by the U.S. Air Force, because it can search large areas of sky and also track very faint (small) objects in and around GEO,  Brian Weeden, a Technical Advisor at the Secure World Foundation, wrote in an email.',
 'The telescope would join another new space debris tracking technology known as the Space Fence, which is now being built by Bethesda-based Lockheed Martin.',
 'Every military operation that takes place in the world today is critically dependent on space in one way or another,  Air Force Gen. John Hyten said in an interview earlier this year when he was the commander of the Air Force Space Command.']